In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.metrics import r2_score
import os
from shapely.geometry import shape
import csv
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import fiona
import rasterio
import rasterio.transform
import rasterio.mask
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

In [2]:
#extracting the datasets

#CALVING FLUX. I take the ones generated by the tif files in the folder 'exploration_ice_calving'
data_dir_ice_calving = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df= pd.read_csv( data_dir_ice_calving+ 'Glacier_ice_calving_withice_shelves_flux_greene.csv', index_col = 0)
calving_df = calving_df.sort_index()
index_calving = calving_df.index.values
calving_columns= calving_df.columns
#create an array with the common years, from 2000 to 2017
common_years = np.arange(2000,2018)
calving = calving_df[common_years.astype(str)]


#BASAL MELTING, retrived from the csv file 'Merged_Integrated_melt_rates.csv'. the index is NOT the same as the calving one
datadir ='/Users/francesco/Desktop/Thesis/Data/'
df_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')
df_err_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_err_rates_second_method.csv')


#Importing the RIGNOT dataset
df_rignot = pd.read_excel('table.xlsx')
df_rignot = df_rignot.set_index('id')
df_rignot = df_rignot.sort_index()

#NB: THE RIGNOT INDEX SHOULD HAVE THE CORRECT VALUES OF THE DF INDEX, BEACUSE I WROTE IT MANUALLY ON THE TABLE EXCEL FILE
df_rignot_index = df_rignot.index.values


/Users/francesco/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
#Extracting the info from the shapefile 'ice_shelf.shp'

shapefile_path = '/Users/francesco/Desktop//Thesis/Data/ice_shelf.shp'

ids = []
Names = []
lat_lon_coords = []
regions = []
areas = []

shapefile=fiona.open(shapefile_path)

print(shapefile.schema['properties'])

for feature in shapefile:

    id = feature['properties']['id']
    name=feature['properties']['name']
    geometry=feature['geometry']
    if geometry is None:
        continue
    region = feature['properties']['regions']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    area = feature['properties']['area']

    bounds=shape(geometry).bounds

    #Taking the info
    ids.append(id)
    Names.append(name)
    lat_lon_coords.append([lat,lon])
    regions.append(region)
    areas.append(area)


{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'float:24.15', 'mass_los16': 'float:24.15', 'mass_los17': 'floa

In [4]:
#Create a dataset with the iformatinos found above
df = pd.DataFrame({ 'Names':Names, 'lat_lon_coords':lat_lon_coords, 'regions':regions, 'area':areas}, index = ids)
#I want to sort the dataset by the names
df = df.sort_values(by=['Names'])
df

,Names,lat_lon_coords,regions,area
110,Abbot,"[-72.50596317, -101.6978671]",West,NaN
111,Abbot_1,"[-71.96090064, -99.79274096]",West,218445628.0
105,Abbot_2,"[-71.99621022, -99.05538844]",West,314087734.0
106,Abbot_3,"[-72.00478146, -98.01541202]",West,318237311.0
107,Abbot_4,"[-71.9475398, -97.24959677]",West,364449396.0
...,...,...,...,...
43,Wordie_(Harriott_Headland),"[-68.99387854, -66.92135035]",Peninsula,36787765.0
41,Wordie_(Prospect),"[-69.40774926, -67.45165471]",Peninsula,162577903.0
82,Wylde,"[-73.57943742, 166.7282965]",East,97818937.0
93,Zelee,"[-66.8119156, 141.2023372]",East,40648302.0


### Calving Comparison Rignot-Me for 2008

In [5]:
#Trying 2008
calving_2008 = calving['2008']

In [6]:
#Selecting the common index between calving and Rignot
common_index_calving_rignot = np.intersect1d(index_calving, df_rignot_index) #indices in common between calving and rignot

# Creating the sub-dataset of rignot and calving which contain just the common indices
df_rignot_common = df_rignot.loc[common_index_calving_rignot]
calving_2008_common = calving_2008.loc[common_index_calving_rignot]


In [7]:
#Create a dataset comparing Calving Rignot-Me

comparison_calving_rignot_me = pd.DataFrame({'name':df_rignot_common.name,'calving_by_me':calving_2008_common, 'calving_rignot':df_rignot['Ice-front flux'], 'percentage': df_rignot['Ice-front flux']/calving_2008_common }, index=common_index_calving_rignot)
comparison_calving_rignot_me
comparison_calving_rignot_me.to_csv('comparison_rignot_calving.csv')

In [8]:
tot_calving_me = calving_2008_common.sum()
tot_calving_rignot = df_rignot_common['Ice-front flux'].sum()

print('Total calving by me: ' , tot_calving_me , ' total calving by rignot: ', tot_calving_rignot)


Total calving by me:  1475.8420447635244  total calving by rignot:  1081.3


### Melting Comparison Rignot-Me

In [32]:
# The melt index is not the correct one. TRYING 2008 NOW    

df_melt_rates
df_melt_rates_2008 = df_melt_rates.loc[:,['Names', '2008']]

In [10]:
#I now select the galciers which are in common between rignot and melt rates, according to the names

common_names_melt_rignot = np.intersect1d(df_melt_rates_2008.Names, df_rignot_common.name)
print(np.shape(common_names_melt_rignot))

#Selecting the common glaciers from df_melt
df_melt_common = df_melt_rates_2008.loc[df_melt_rates.Names.isin(common_names_melt_rignot)]
df_melt_common #Till here the values are correct


(61,)


,Names,2008
5,LarsenD,9.70
7,LarsenC,81.18
9,WilmaRobertDowner,0.60
10,Shirase,6.72
12,Rayner_Thyer,2.71
...,...,...
167,Holmes,-3.46
168,Moscow_University,28.06
169,Prince_Harald,-7.35
175,LarsenE,6.12


In [11]:
#Rebuilding the index for the d_f_melt_rates_common
df_melt_index = []
df_melt_names_common_temp = []
df_melt_melt_temp = []
df_melt_rignot_temp = []
df_melt_rignot_temp_Gton = []



#In this cycle i check when the name in the 2 dataset are the same, in which case I save the name,  the melt by Rignot and by me and the index in Rignot

j=0
for name_rignot in df_rignot_common.name:
    i =0
    for name_melt in df_melt_common.Names:

        if name_melt == name_rignot:
            df_melt_index.append(df_rignot_common.index[j]) 
            df_melt_names_common_temp.append(name_melt)
            df_melt_melt_temp.append(df_melt_common['2008'].iloc[i])
            df_melt_rignot_temp.append(df_rignot_common['Basal melting per meter m/yr'].iloc[j])
            df_melt_rignot_temp_Gton.append(df_rignot_common['Basal melting tot Gt/yr'].iloc[j])
            
        i = i+1

    j = j+1

In [12]:
melt_comparison_rignot_me = pd.DataFrame({'name':df_melt_names_common_temp, 'melt_by_me':df_melt_melt_temp, 'melt_by_rignot_Gt_yr': df_melt_rignot_temp_Gton, 'ratio_rignot_me': np.array(df_melt_rignot_temp_Gton)/np.array(df_melt_melt_temp)}, index=df_melt_index)
#i want to sort the dataframe by index
melt_comparison_rignot_me = melt_comparison_rignot_me.sort_index()
melt_comparison_rignot_me
melt_comparison_rignot_me.to_csv('comparison_rignot_melting.csv')


In [13]:
tot_melt_me = np.array(df_melt_melt_temp).sum()
tot_melt_rignot = np.array(df_melt_rignot_temp_Gton).sum()

print('Total melt by me: ' , tot_melt_me , ' total melt by rignot: ', tot_melt_rignot)

Total melt by me:  1554.1200000000001  total melt by rignot:  1310.2000000000003


### I now want to do a comparison between the calving flux and the melt flux

In [14]:
#Here I go through the calving dataset and I add the names where there is the correct index
for index_value in calving.index:

    name = df.loc[index_value, 'Names']
    calving.loc[index_value, 'Names'] = name

calving

/var/folders/9n/18cd5sld4lv1jvk0nw84p_bh0000gq/T/ipykernel_61443/3768571545.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calving.loc[index_value, 'Names'] = name


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Names
1,0.999649,0.313843,0.790420,0.732301,0.000000,0.836916,0.499825,0.232477,0.848539,0.000000,0.000000,0.557944,0.337091,0.418458,0.464953,0.000000,0.220853,0.069743,Dawson_Lambton
2,0.370000,0.000000,0.290000,0.210000,0.000000,0.320000,0.000000,0.020000,1.170000,0.000000,0.310000,0.000000,0.050000,0.110000,0.180000,0.000000,0.000000,0.000000,Philbin_Inlet
3,0.000000,0.000000,0.149756,0.138236,0.000000,0.138236,0.161276,0.149756,0.011520,0.000000,0.138236,0.000000,0.000000,0.138236,0.126717,0.000000,0.000000,0.000000,Walgreen_Coast_2
4,2.910863,0.286153,0.868325,0.740050,0.000000,0.000000,0.029602,0.059204,1.430763,0.000000,0.000000,0.069071,0.374959,0.601907,0.256551,0.000000,0.000000,0.000000,Jackson
5,0.285593,0.118176,0.147720,0.009848,0.000000,0.206808,0.147720,0.167416,0.295441,0.118176,0.196960,0.118176,0.177264,0.177264,0.216656,0.226504,0.226504,0.216656,LarsenD_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,0.260000,0.000000,0.140000,0.040000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.040000,0.040000,Britten
177,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ClarkeBay
178,0.000000,0.000000,0.372028,0.420979,0.283916,0.000000,0.362238,0.156643,0.000000,0.000000,0.303496,0.558042,0.117482,0.450349,0.009790,0.000000,0.048951,0.029371,Hayes_Coats_Coast
180,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,Rydberg_Peninsula_1


In [15]:
#Basal Melting USED, retrived from the csv file 'Merged_Integrated_melt_rates.csv'. 

#I had changed manually the index in the excel file, looking in the iceshelf_flux_greene.gpkg file, checking the attibute table

datadir ='/Users/francesco/Desktop/Thesis/Data/'
melt_rates = pd.read_excel(datadir + '/Merged_Integrated_melt_rates_with_id_manually_added.xlsx', index_col=0)
melt_rates.sort_index(inplace=True)

In [16]:
#Check if there are repetead values in the index of the dataset df_melt_rates
melt_rates.index.is_unique

#What are the repeated numbers?
unique, counts = np.unique(melt_rates.index, return_counts=True)

# print the repeated values
print(unique[counts > 1])

[nan]


In [17]:
#Working on the compatibility of the 2 datasets, calving from tif and melt rates from the csv file

#Selecting the common index, and converting the datasets
common_index_calving_melting = np.intersect1d(calving.index, melt_rates.index)

melt_rates_common = melt_rates.loc[common_index_calving_melting]
calving_common = calving.loc[common_index_calving_melting]

#Moving the names to the first column in the calving dataset
names_temp = calving_common.pop('Names')
calving_common.insert(0, 'Names', names_temp)

#Finding the common columns, and converting them to strings (in order to do the comparison)
cols1 = np.array(melt_rates_common.columns) #columns of the melt rates
cols2 = np.array(calving_common.columns) #columns of the calving dataset
melt_rates_common.columns.astype(str)
common_cols = np.intersect1d(cols1.astype(str), cols2)

#Convert the columns to string
melt_rates_common.columns = melt_rates_common.columns.astype(str)

#creating the subset of melt_rates, with common columns
melt_rates_common = melt_rates_common.loc[:,common_cols]
names_temp = melt_rates_common.pop('Names')
melt_rates_common.insert(0, 'Names', names_temp)


In [18]:
years = np.arange(2000, 2018, 1)
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [19]:
ids = []
Names = []
melt = []
calving = []
tot =[]

Years = np.arange(2000, 2018, 1)

for index_value in melt_rates_common.index:
    ids.append(index_value)
    Names.append(melt_rates_common.loc[index_value, 'Names'])

    #For melt rates common, caving and total i add the whole time serie
    melt.append(melt_rates_common.loc[index_value, Years.astype(str)].values)
    calving.append(calving_common.loc[index_value, Years.astype(str)].values)
    tot.append(melt_rates_common.loc[index_value, Years.astype(str)].values + calving_common.loc[index_value, Years.astype(str)].values)

         

In [20]:
#Dataset with ids and time_series

df_comparison_melting_calving_final = pd.DataFrame({'Names':Names, 'melt_time_serie':melt, 'calving_time_serie':calving, 'tot':tot}, index=ids)


In [21]:
from matplotlib.backends.backend_pdf import PdfPages

''' 

with PdfPages('Glaciers calving and melting comparison.pdf') as pdf:

    for id in df_comparison_melting_calving_final.index:

        fig = plt.figure()

        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['melt_time_serie'],label =  ' Melting rate ')
        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['calving_time_serie'],label = ' Calving')
        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['tot'],label = ' tot')
        

        plt.legend()
        plt.xlabel('Years')
        plt.ylabel('Ice Loss (Gt/yr)')

        #In the title I want to print the R2 value, rounded with 2 decimals
        plt.title('Melting rates of galcier ' + str(df_comparison_melting_calving_final.loc[id]['Names']) + ', its id is: ' + str(id) )




        pdf.savefig(fig) 
        plt.close()
'''

" \n\nwith PdfPages('Glaciers calving and melting comparison.pdf') as pdf:\n\n    for id in df_comparison_melting_calving_final.index:\n\n        fig = plt.figure()\n\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['melt_time_serie'],label =  ' Melting rate ')\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['calving_time_serie'],label = ' Calving')\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['tot'],label = ' tot')\n        \n\n        plt.legend()\n        plt.xlabel('Years')\n        plt.ylabel('Ice Loss (Gt/yr)')\n\n        #In the title I want to print the R2 value, rounded with 2 decimals\n        plt.title('Melting rates of galcier ' + str(df_comparison_melting_calving_final.loc[id]['Names']) + ', its id is: ' + str(id) )\n\n\n\n\n        pdf.savefig(fig) \n        plt.close()\n"

In [22]:
#Problems with Wordie_(Harriott) AND Wordie_(Harriott_Headland). But the correction I had done is right, the problem is probably on claving dataset

#Check if they are unique
print(melt_rates_common.index.is_unique)
print(calving_common.index.is_unique)

#Check if they have the same index
if np.array_equal(melt_rates_common['Names'], calving_common['Names']):
    print('The two dataframes have the same index')
else:
    print('The two dataframes have different index')

#spot which is the element which is different
for i in range(len(melt_rates_common['Names'])):
    if melt_rates_common['Names'].iloc[i] != calving_common['Names'].iloc[i]:
        print('The element which is different is: ', melt_rates_common['Names'].iloc[i], ' and it is in position: ', i)



True
True
The two dataframes have different index
The element which is different is:  Wordie_(Harriott)  and it is in position:  42
The element which is different is:  Wordie_(Harriott_Headland)  and it is in position:  43


In [23]:
### Check the results with GREEN (calving) AND PAOLO (melting)

### Check the results with Green paper

In [36]:
#CALVING FLUX. I take the ones generated by the shp files in the folder 'exploration_ice_calving'
#IN this shp files the negative values were put to 0
data_dir_ice_calving = '/Users/francesco/Desktop/Thesis/map_bm_and_ic/'
#open the file Dataset_Green_modified.xlsx, which is in this directory
calving_df = pd.read_excel(data_dir_ice_calving + 'Dataset_Green_modified.xlsx', index_col=0)


calving_df = calving_df.sort_index()
index_calving = calving_df.index.values

calving_df

,Names,1997.75,2000.2,2000.75,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
ids,,,,,,,,,,,,,,,,,,,,,,,,,
1,Dawson Lambton,0,0.860168,-2.152202,0.274296,0.682000,0.632936,-0.072896,0.717127,0.434690,0.204823,0.731130,-0.363177,-0.051922,0.480620,0.292687,0.363594,0.401445,-0.743185,0.187785,0.059674,0.128567,0.058141,0.049661,-0.030599
2,Philbin Inlet,0,0.368595,-0.691326,-0.336273,0.289940,0.214561,-0.659199,0.316612,-0.614449,0.023739,1.167137,-0.893040,0.311802,-0.614449,0.049995,0.109901,0.177765,-0.346653,0.000000,0.000000,0.000000,0.000000,0.006645,0.000000
3,Walgreen Coast 2,0,-0.631544,-0.297324,-0.407974,0.129289,0.120999,-0.354887,0.122751,0.141737,0.130596,0.010983,-0.400410,0.122767,-0.295681,-0.056309,0.121592,0.113173,-0.434267,0.000000,0.000000,0.000000,0.019536,-0.020696,-0.001969
4,Jackson,0,2.949785,-0.835735,0.287100,0.878513,0.754236,-0.323696,-0.686876,0.026670,0.064556,1.451946,-0.729172,-1.382473,0.067120,0.382227,0.607256,0.260380,-1.395914,-0.099928,0.000000,0.000000,0.026449,0.025557,0.000000
5,Larsen D 1,0,0.287605,-0.287605,0.117748,0.154905,0.014952,-0.277849,0.208921,0.147554,0.169760,0.295675,0.124332,0.204939,0.122477,0.177277,0.179321,0.221109,0.226448,0.226810,0.216833,0.217167,0.219927,0.215649,-2.030783
6,Larsen D,0,10.522705,-8.450439,-0.420410,5.223877,4.860107,-0.992432,4.797607,3.755615,3.912354,5.427490,1.854980,4.361816,3.833496,2.974854,3.435547,-0.658447,6.263916,9.036621,5.920166,5.799072,5.518555,5.284180,-97.033447
7,Larsen F,0,-13.045746,-9.180267,0.675613,1.136063,1.092377,0.166534,1.088730,0.912109,0.933075,1.193237,0.643356,0.843170,0.662155,0.635498,0.628189,-1.073822,-0.989655,0.426559,0.322617,0.325790,0.162704,0.035309,0.031891
8,Larsen C,0,69.914062,-2.210938,-21.234375,27.066406,18.739258,-5.194336,-293.833984,15.952148,33.801758,49.739258,1.317383,2.911133,31.314453,34.605469,43.868164,5.268555,32.607422,28.774414,33.502930,-1285.419922,7.253906,-11.208008,15.540039
9,Telen,0,-0.331386,0.071259,-3.248991,0.096731,0.019883,-0.011875,-0.012002,0.041574,-0.024470,-0.022391,0.025497,-0.018096,0.037211,-0.036318,-0.056778,-0.002242,-0.053013,-0.016677,0.007537,0.005250,0.048833,-0.407024,-0.088769


In [25]:
#Calculating the total loss from the calving fluxes from the shp files



-1807.4199999999998

In [26]:
ids_roi = [56,8,34,70,35]

In [42]:
calving_df_cumsum_roi = calving_df.loc[ids_roi]
#Add a clumns tot at the end of the dataframe, which is the sum of all the columns
calving_df_cumsum_roi['tot'] = calving_df_cumsum_roi.sum(axis=1)
calving_df_cumsum_roi

/var/folders/9n/18cd5sld4lv1jvk0nw84p_bh0000gq/T/ipykernel_61443/461474375.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  calving_df_cumsum_roi['tot'] = calving_df_cumsum_roi.sum(axis=1)


,Names,1997.75,2000.2,2000.75,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,tot
ids,,,,,,,,,,,,,,,,,,,,,,,,,,
56,Thwaites,0,0.394287,-70.696777,41.095459,-118.523682,0.586182,-120.988037,67.379150,51.924072,43.535645,52.743896,19.632812,-253.186523,-37.443115,-475.821777,-802.573853,-69.577515,-16.321655,-64.994019,-41.280762,-47.362427,-6.858643,2.826904,-122.895630,-1968.406006
8,Larsen C,0,69.914062,-2.210938,-21.234375,27.066406,18.739258,-5.194336,-293.833984,15.952148,33.801758,49.739258,1.317383,2.911133,31.314453,34.605469,43.868164,5.268555,32.607422,28.774414,33.502930,-1285.419922,7.253906,-11.208008,15.540039,-1166.924805
34,Ronne,0,-1269.234375,-2511.312500,-13.765625,151.031250,153.750000,124.500000,148.125000,98.765625,194.156250,166.078125,54.000000,165.515625,162.968750,162.406250,153.796875,125.000000,134.437500,113.609375,130.000000,128.203125,133.218750,140.468750,122.671875,-1031.609375
70,Ross West,0,-2839.960938,-57.671875,39.937500,117.335938,108.992188,3.218750,79.343750,99.382812,69.234375,189.156250,38.359375,83.421875,127.273438,86.632812,116.890625,91.742188,104.554688,95.398438,117.546875,96.820312,113.437500,107.859375,81.476562,-929.617188
35,Filchner,0,207.742188,37.992188,-47.273438,84.437500,83.328125,44.445312,55.726562,85.429688,83.375000,113.414062,24.039062,75.492188,93.296875,78.945312,88.914062,60.234375,95.820312,84.390625,89.125000,91.031250,85.625000,95.281250,85.445312,1796.257812


###Trying with the tif dataset

In [30]:
#CALVING FLUX. I take the ones generated by the tif files in the folder 'exploration_ice_calving'
data_dir_ice_calving_tif = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df_tif= pd.read_csv( data_dir_ice_calving_tif+ '/Glacier_ice_calving_with_ice_shelves_flux_greene.csv', index_col = 0)
calving_df_tif = calving_df.sort_index()
index_calving_tif = calving_df_tif.index.values

#calving_df.insert(0, 'Names', df['Names'])
calving_df_tif


#I want to calculate the cumulative sum of the calving fluxes
#calving_df_cumsum = calving_df.cumsum(axis=1)

#print(calving_df)
#calving_df_cumsum = calving_df.cumsum(axis=1)
#Do the cumulative sum of the calving fluxes, excluding the first column Name
#calving_df_tif_cumsum = calving_df_tif.iloc[:,1:].cumsum(axis=1)
#calving_df_tif_cumsum.insert(0, 'Names', df['Names'])



,Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
id_measures,,,,,,,,,,,,,,,,,,,,,,,
1,Dawson_Lambton,0.86,0.27,0.68,0.63,-0.07,0.72,0.43,0.20,0.73,-0.36,-0.05,0.48,0.29,0.36,0.40,-0.74,0.19,0.06,0.13,0.06,0.05,-0.03
2,Philbin_Inlet,0.37,-0.34,0.29,0.21,-0.66,0.32,-0.61,0.02,1.17,-0.89,0.31,-0.61,0.05,0.11,0.18,-0.35,0.00,0.00,0.00,0.00,0.01,0.00
3,Walgreen_Coast_2,-0.63,-0.41,0.13,0.12,-0.35,0.12,0.14,0.13,0.01,-0.40,0.12,-0.30,-0.06,0.12,0.11,-0.43,0.00,0.00,0.00,0.02,-0.02,0.00
4,Jackson,2.95,0.29,0.88,0.75,-0.32,-0.69,0.03,0.06,1.45,-0.73,-1.38,0.07,0.38,0.61,0.26,-1.40,-0.10,0.00,0.00,0.03,0.03,0.00
5,LarsenD_1,0.29,0.12,0.15,0.01,-0.28,0.21,0.15,0.17,0.30,0.12,0.20,0.12,0.18,0.18,0.22,0.23,0.23,0.22,0.22,0.22,0.22,-2.03
6,LarsenD,10.52,-0.42,5.22,4.86,-0.99,4.80,3.76,3.91,5.43,1.85,4.36,3.83,2.97,3.44,-0.66,6.26,9.04,5.92,5.80,5.52,5.28,-97.03
7,LarsenF,-13.05,0.68,1.14,1.09,0.17,1.09,0.91,0.93,1.19,0.64,0.84,0.66,0.64,0.63,-1.07,-0.99,0.43,0.32,0.33,0.16,0.04,0.03
8,LarsenC,69.91,-21.23,27.07,18.74,-5.19,-293.83,15.95,33.80,49.74,1.32,2.91,31.31,34.61,43.87,5.27,32.61,28.77,33.50,-1285.42,7.25,-11.21,15.54
9,Telen,-0.33,-3.25,0.10,0.02,-0.01,-0.01,0.04,-0.02,-0.02,0.03,-0.02,0.04,-0.04,-0.06,0.00,-0.05,-0.02,0.01,0.01,0.05,-0.41,-0.09


In [31]:
calving_df_tif

,Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
id_measures,,,,,,,,,,,,,,,,,,,,,,,
1,Dawson_Lambton,0.86,0.27,0.68,0.63,-0.07,0.72,0.43,0.20,0.73,-0.36,-0.05,0.48,0.29,0.36,0.40,-0.74,0.19,0.06,0.13,0.06,0.05,-0.03
2,Philbin_Inlet,0.37,-0.34,0.29,0.21,-0.66,0.32,-0.61,0.02,1.17,-0.89,0.31,-0.61,0.05,0.11,0.18,-0.35,0.00,0.00,0.00,0.00,0.01,0.00
3,Walgreen_Coast_2,-0.63,-0.41,0.13,0.12,-0.35,0.12,0.14,0.13,0.01,-0.40,0.12,-0.30,-0.06,0.12,0.11,-0.43,0.00,0.00,0.00,0.02,-0.02,0.00
4,Jackson,2.95,0.29,0.88,0.75,-0.32,-0.69,0.03,0.06,1.45,-0.73,-1.38,0.07,0.38,0.61,0.26,-1.40,-0.10,0.00,0.00,0.03,0.03,0.00
5,LarsenD_1,0.29,0.12,0.15,0.01,-0.28,0.21,0.15,0.17,0.30,0.12,0.20,0.12,0.18,0.18,0.22,0.23,0.23,0.22,0.22,0.22,0.22,-2.03
6,LarsenD,10.52,-0.42,5.22,4.86,-0.99,4.80,3.76,3.91,5.43,1.85,4.36,3.83,2.97,3.44,-0.66,6.26,9.04,5.92,5.80,5.52,5.28,-97.03
7,LarsenF,-13.05,0.68,1.14,1.09,0.17,1.09,0.91,0.93,1.19,0.64,0.84,0.66,0.64,0.63,-1.07,-0.99,0.43,0.32,0.33,0.16,0.04,0.03
8,LarsenC,69.91,-21.23,27.07,18.74,-5.19,-293.83,15.95,33.80,49.74,1.32,2.91,31.31,34.61,43.87,5.27,32.61,28.77,33.50,-1285.42,7.25,-11.21,15.54
9,Telen,-0.33,-3.25,0.10,0.02,-0.01,-0.01,0.04,-0.02,-0.02,0.03,-0.02,0.04,-0.04,-0.06,0.00,-0.05,-0.02,0.01,0.01,0.05,-0.41,-0.09
